In [1]:
!pip install neuralforecast datasets openmeteo-requests pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.2/263.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.1/167.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.2/683.2 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install neuralforecast openmeteo-requests requests-cache retry-requests pandas matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 7.8 MB/s eta 0:00:00


In [3]:
import requests
import pandas as pd
import numpy as np

params_weather = {
    "latitude": 56.0, "longitude": 3.0,
    "start_date": "2023-01-01", "end_date": "2024-01-01",
    "hourly": ["pressure_msl", "wind_speed_10m", "wind_direction_10m"]
}
url_weather = "https://archive-api.open-meteo.com/v1/archive"
response_weather = requests.get(url_weather, params=params_weather).json()

df_weather = pd.DataFrame({
    'date': pd.to_datetime(response_weather['hourly']['time']),
    'pressure': response_weather['hourly']['pressure_msl'],
    'wind_speed': response_weather['hourly']['wind_speed_10m'],
    'wind_dir': response_weather['hourly']['wind_direction_10m']
})

df_weather['wind_sin'] = np.sin(np.deg2rad(df_weather['wind_dir']))
df_weather['wind_cos'] = np.cos(np.deg2rad(df_weather['wind_dir']))

params_marine = {
    "latitude": 56.0, "longitude": 3.0,
    "start_date": "2023-01-01", "end_date": "2024-01-01",
    "hourly": ["wave_height"]
}
url_marine = "https://marine-api.open-meteo.com/v1/marine"
response_marine = requests.get(url_marine, params=params_marine).json()

df_marine = pd.DataFrame({
    'date': pd.to_datetime(response_marine['hourly']['time']),
    'y': response_marine['hourly']['wave_height']
})

df = pd.merge(df_weather, df_marine, on="date")
df = df.rename(columns={'date': 'ds'})
df['unique_id'] = 'NorthSea_Point1'
df = df.dropna()

print("data processing completed successfully. Now we have wind vectors sin/cos")

data processing completed successfully. Now we have wind vectors sin/cos


# NHITS